<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

In [1]:
import skimage
from skimage import io
import matplotlib.pyplot as plt
import json
import cv2
import numpy as np
import matplotlib
import os

In [34]:
DATA_DIR= "./data/"

def integrate_parallel(dataset = 'train'):

    mask = skimage.io.imread(DATA_DIR+'mask.png')
    if not os.path.isdir(DATA_DIR+'integrated_imgs_' + dataset):
        os.makedirs(DATA_DIR+'integrated_imgs_' + dataset)
    for folder in os.listdir(DATA_DIR + dataset):
        if folder.startswith(dataset) or folder.startswith('valid'):
            if not os.path.isdir(DATA_DIR + '/integrated_imgs_' + dataset + '/' + folder):
                os.makedirs(DATA_DIR + '/integrated_imgs_' + dataset + '/' + folder)
            with open(DATA_DIR + dataset + '/' + folder + '/homographies.json') as f:
                homographies = json.load(f)
            for time_point in [0,1,2,3,4,5,6]:
                warped_imgs = []
                for camera in ['B01', 'B02', 'B03', 'B04', 'B05', 'G01', 'G02', 'G03', 'G04', 'G05']:
                    img = skimage.io.imread(DATA_DIR + dataset + '/' + folder + '/' + str(time_point) + '-' + camera + '.png')
                    img = img[..., :3].astype(np.float32)/255
                    img[mask==0] = np.nan

                    homography = np.array(homographies[str(time_point) + '-' + camera])
                    warped_img = img.copy()
                    for dim in range(3):
                        warped_img[:,:,dim] = cv2.warpPerspective(img[:,:,dim],homography,img.shape[:2],borderMode = cv2.BORDER_CONSTANT,borderValue = np.nan) 
                    warped_imgs.append(warped_img)

                integrated_img = np.nanmean(np.array(warped_imgs), axis = 0)

                integrated_img_nan = integrated_img.copy()
                for dim in range(3):
                    integrated_img_nan[:,:,dim] = np.where(np.isnan(warped_imgs).any(axis = 0).all(axis = 2), np.nan, integrated_img[:,:,dim])

            #     plt.figure()
            #     plt.imshow(integrated_img)
            #     plt.figure()
            #     plt.imshow(integrated_img_nan)
            
                matplotlib.image.imsave(DATA_DIR + '/integrated_imgs_' + dataset + '/' + folder + '/' + str(time_point) + '-' + camera + '_integrated.png', integrated_img)
                matplotlib.image.imsave(DATA_DIR + '/integrated_imgs_' + dataset + '/' + folder + '/' + str(time_point) + '-' + camera + '_integrated_nan.png', integrated_img_nan)


In [35]:
def integrate_all(dataset = 'train'):

    mask = skimage.io.imread(DATA_DIR + 'mask.png')


    new_path = DATA_DIR + 'integrated_imgs_' + dataset



    if not os.path.isdir(new_path):
        print('create dir: ', new_path)
        os.makedirs(new_path)
    else:
        print(new_path, ' exists')

    for folder in os.listdir(DATA_DIR + dataset):
        if folder.startswith(dataset) or folder.startswith('valid'):
            print(f'perform merging of images in {folder} in {DATA_DIR + dataset}')
            if not os.path.isdir(new_path + '/' + folder):
                print('create dir: ', new_path)
                os.makedirs(new_path + '/' + folder)
            with open(DATA_DIR + '' + dataset + '/' + folder + '/homographies.json') as f:
                homographies = json.load(f)

            warped_imgs = []
            for time_point in [0,1,2,3,4,5,6]:
                for camera in ['B01', 'B02', 'B03', 'B04', 'B05', 'G01', 'G02', 'G03', 'G04', 'G05']:
                    img = skimage.io.imread(DATA_DIR + '' + dataset + '/' + folder + '/' + str(time_point) + '-' + camera + '.png')
                    img = img[..., :3].astype(np.float32)/255
                    img[mask==0] = np.nan

                    homography = np.array(homographies[str(time_point) + '-' + camera])
                    warped_img = img.copy()
                    for dim in range(3):
                        warped_img[:,:,dim] = cv2.warpPerspective(img[:,:,dim],homography,img.shape[:2],borderMode = cv2.BORDER_CONSTANT,borderValue = np.nan) 
                    warped_imgs.append(warped_img)

            integrated_img = np.nanmean(np.array(warped_imgs), axis = 0)

            integrated_img_nan = integrated_img.copy()
            for dim in range(3):
                integrated_img_nan[:,:,dim] = np.where(np.isnan(warped_imgs).any(axis = 0).all(axis = 2), np.nan, integrated_img[:,:,dim])

    #         plt.figure()
    #         plt.imshow(integrated_img)
    #         plt.figure()
    #         plt.imshow(integrated_img_nan)

            print('save ', new_path + '/' + folder + '/all_integrated*', integrated_img)
            matplotlib.image.imsave(new_path + '/' + folder + '/all_integrated.png', integrated_img)
            matplotlib.image.imsave(new_path + '/' + folder + '/all_integrated_nan.png', integrated_img_nan)
        else:
            print(f'did not perform merging of images in {folder} in {DATA_DIR + dataset} because'
                  f'{DATA_DIR + dataset} does not start with {folder}')

In [36]:
integrate_parallel('train')
integrate_parallel('test')
integrate_parallel('validation')

integrate_all('train')
integrate_all('test')
integrate_all('validation')

/var/folders/g8/7dsc7c7d5757b7m6xbqnwy680000gn/T/ipykernel_94893/2146053789.py:27: RuntimeWarning: Mean of empty slice
  integrated_img = np.nanmean(np.array(warped_imgs), axis = 0)


./data/integrated_imgs_train  exists
perform merging of images in train-1-5 in ./data/train


/var/folders/g8/7dsc7c7d5757b7m6xbqnwy680000gn/T/ipykernel_94893/1647082242.py:38: RuntimeWarning: Mean of empty slice
  integrated_img = np.nanmean(np.array(warped_imgs), axis = 0)


save  ./data/integrated_imgs_train/train-1-5/all_integrated* [[[nan nan nan]
  [nan nan nan]
  [nan nan nan]
  ...
  [nan nan nan]
  [nan nan nan]
  [nan nan nan]]

 [[nan nan nan]
  [nan nan nan]
  [nan nan nan]
  ...
  [nan nan nan]
  [nan nan nan]
  [nan nan nan]]

 [[nan nan nan]
  [nan nan nan]
  [nan nan nan]
  ...
  [nan nan nan]
  [nan nan nan]
  [nan nan nan]]

 ...

 [[nan nan nan]
  [nan nan nan]
  [nan nan nan]
  ...
  [nan nan nan]
  [nan nan nan]
  [nan nan nan]]

 [[nan nan nan]
  [nan nan nan]
  [nan nan nan]
  ...
  [nan nan nan]
  [nan nan nan]
  [nan nan nan]]

 [[nan nan nan]
  [nan nan nan]
  [nan nan nan]
  ...
  [nan nan nan]
  [nan nan nan]
  [nan nan nan]]]
perform merging of images in train-1-2 in ./data/train
save  ./data/integrated_imgs_train/train-1-2/all_integrated* [[[nan nan nan]
  [nan nan nan]
  [nan nan nan]
  ...
  [nan nan nan]
  [nan nan nan]
  [nan nan nan]]

 [[nan nan nan]
  [nan nan nan]
  [nan nan nan]
  ...
  [nan nan nan]
  [nan nan nan]
  [

In [33]:
# integrate_all('train')

integrate_all('validation')

./data/integrated_imgs_validation  exists
did not perform merging of images in labels.json in ./data/validation because./data/validation does not start with labels.json
perform merging of images in valid-2-0 in ./data/validation
create dir:  ./data/integrated_imgs_validation


KeyboardInterrupt: 